# Programming Assignment: Размещение баннеров

Представим, что международное круизное агентство *Carnival Cruise Line* решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.  

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.  

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org: https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив *fsq.zip* с этой страницы. Нас будет интересовать файл checkins.dat.

In [1]:
import pandas as pd
import numpy as np

data = []

with open('checkins.dat') as f:
    columns = list(map(str.strip, f.readline().strip().split('|'))) # get a list of columns
    for line in f.readlines():
        data_row = list(map(str.strip, line.strip().split('|')))
        if len(data_row) == len(columns): # check if this row has proper n of columns
            data.append(data_row)
df_raw = pd.DataFrame(data)  
df_raw.columns = columns

Открыв его, увидим следующую структуру:

In [2]:
df_raw.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984301,2041916,5222,,,2012-04-21 17:39:01
1,984222,15824,5222,38.8951118,-77.0363658,2012-04-21 17:43:47
2,984315,1764391,5222,,,2012-04-21 17:37:18
3,984234,44652,5222,33.800745,-84.41052,2012-04-21 17:43:43
4,984249,2146840,5222,,,2012-04-21 17:42:58


Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас:

In [3]:
df = df_raw[(df_raw.latitude != '') & (df_raw.longitude != '')] # drop with empty coordinates
df.head()

,id,user_id,venue_id,latitude,longitude,created_at
1,984222,15824,5222,38.8951118,-77.0363658,2012-04-21 17:43:47
3,984234,44652,5222,33.800745,-84.41052,2012-04-21 17:43:43
7,984291,105054,5222,45.5234515,-122.6762071,2012-04-21 17:39:22
9,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
10,984232,93870,380645,33.4483771,-112.0740373,2012-04-21 17:38:18


Убедимся, что все 396634 строки с координатами считаны успешно.

In [4]:
print(df.shape)

(396634, 6)


Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом [MeanShift](http://scikit-learn.org/stable/modules/clustering.html#mean-shift), который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

**Примечание**: на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [5]:
%%time
from sklearn.cluster import MeanShift

coordinates = df[['latitude', 'longitude']] # need only coordinates
coordinates_samples = coordinates.sample(100000) # get random 100000 rows

ms = MeanShift(bandwidth=0.1, n_jobs=-1)
ms.fit(coordinates_samples)

CPU times: user 51.4 s, sys: 5.4 s, total: 56.8 s
Wall time: 2min 35s


In [6]:
coordinates_samples.shape # 100000 random checkins in different places

(100000, 2)

In [7]:
coordinates_samples[:5]

,latitude,longitude
50946,40.65,-73.95
618698,41.8781136,-87.6297982
378260,38.8951118,-77.0363658
658716,37.4282724,-121.9066238
671266,34.1435878,-118.3952221


In [8]:
ms.cluster_centers_.shape # we got 3592 clusters in total

(3614, 2)

In [9]:
ms.cluster_centers_[:5] # coordinates of clusters' centers

array([[  40.71749079,  -73.98961297],
       [  41.87814526,  -87.62982558],
       [  33.44846438, -112.07410016],
       [  33.44624229, -111.90185275],
       [  38.88629231,  -77.04853713]])

In [10]:
unique, counts = np.unique(ms.labels_, return_counts=True) # calc n of points in each center

df_clusters = pd.DataFrame(columns=['cent_latitude', 'cent_longitude', 
                                    'clust_size', 'min_distance'])

df_clusters.cent_latitude = ms.cluster_centers_[:,0]
df_clusters.cent_longitude = ms.cluster_centers_[:,1]
df_clusters.clust_size = counts

In [11]:
df_clusters = df_clusters[df_clusters.clust_size > 15] # get only clasters with > 15 points
df_clusters[:10]

,cent_latitude,cent_longitude,clust_size,min_distance
0,40.717491,-73.989613,14311,NaN
1,41.878145,-87.629826,3934,NaN
2,33.448464,-112.074100,2823,NaN
3,33.446242,-111.901853,2292,NaN
4,38.886292,-77.048537,2746,NaN
5,37.687228,-122.409044,3760,NaN
6,33.767588,-84.393062,1831,NaN
7,42.363248,-71.074186,1633,NaN
8,47.606201,-122.332057,1471,NaN
9,34.066156,-118.268165,912,NaN


При желании увидеть получившиеся результаты на карте можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт [mapcustomizer.com](https://www.mapcustomizer.com/) имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров в формате:

```
38.8951118, -77.0363658

33.800745, -84.41052

45.5234515, -122.6762071
...
```

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

```
33.751277, -118.188740 (Los Angeles)

25.867736, -80.324116 (Miami)

51.503016, -0.075479 (London)

52.378894, 4.885084 (Amsterdam)

39.366487, 117.036146 (Beijing)

-33.868457, 151.205134 (Sydney)
```

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [12]:
# offices coordinates
offices = [[33.751277, -118.188740],
           [25.867736, -80.324116],
           [51.503016, -0.075479],
           [52.378894, 4.885084],
           [39.366487, 117.036146],
           [-33.868457, 151.205134]]

def calc_distance(point1, point2):
    return ((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2) ** (1 / 2)

for i, cluster in enumerate(df_clusters.values):
    temp_distances = [] # create a temp list to store distance from cluster to each office
    for office in offices:
        temp_distances.append(calc_distance([cluster[0], cluster[1]], office))
    
    df_clusters.iloc[i,3] = min(temp_distances)

In [13]:
df_clusters.to_csv('clusters.csv', index=False)

In [14]:
top_clusters = df_clusters.sort_values(by='min_distance')[:20] # get top20 clusters
top_clusters

,cent_latitude,cent_longitude,clust_size,min_distance
273,-33.865455,151.205026,47,0.00300391
283,52.372809,4.892478,46,0.00957642
321,25.842490,-80.315828,40,0.0265716
51,51.503516,-0.126634,306,0.0511574
50,33.812629,-118.144897,268,0.0754071
23,25.786088,-80.214925,706,0.136342
102,26.003775,-80.212508,136,0.175963
80,33.896504,-118.063185,162,0.191976
35,33.868063,-118.367325,452,0.213382
48,33.642865,-117.945832,325,0.266003


In [15]:
# find the closest claster to any office
closest_cluster = top_clusters.head(n=1)
closest_cluster

,cent_latitude,cent_longitude,clust_size,min_distance
273,-33.865455,151.205026,47,0.00300391


In [16]:
def write_answer1(cluster):
    with open('answer1.txt', 'w') as f:
        f.write('{} {}'.format(closest_cluster.iloc[0,0], closest_cluster.iloc[0,1]))
    !cat answer1.txt
    
write_answer1(closest_cluster)

-33.86545504680849 151.20502565744692